In [ ]:
# %load ../jupyter_setup.txt
# Convenient jupyter setup
%load_ext autoreload
%autoreload 2
%config IPCompleter.greedy=True
%config IPCompleter.use_jedi=False 

# 1. Imports

In [ ]:
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd

import rioxarray as rxr
import geopandas as gpd

import pylandstats as pls

from src.geograph import GeoGraph
from src.constants import DATA_DIR, UTM35N, ROIS

# Parse geotif landcover data
chernobyl_path = lambda year: DATA_DIR / "chernobyl" / "esa_cci" / f"esa_cci_{year}_chernobyl.tif" 

# Parse ROIS
rois = gpd.read_file(ROIS)
cez = rois[rois["name"] == "Chernobyl Exclusion Zone"]

# 2. Loading Chernobyl Landcover data (ESA CCI)

For this demonstration we will use the ESA CCI landcover dataset as an example to illustrate how
geographs enable us to reproduce the same metrics that traditional fragmentation software (pylandstats, fragstats)
produce.

In this demo we will look specifically at the Chernobyl exclusion zone, so we reproject the satellite data to the right coordinate
system (UTM35N) and clip to the CEZ region of interest).

In [ ]:
def clip_and_reproject(xrdata, clip_geometry=None, to_crs=UTM35N, x_res=300, y_res=300):
    
    if clip_geometry is not None:
        clipped_data = xrdata.rio.clip(clip_geometry)
    else:
        clipped_data = xrdata
        
    if to_crs is not None:
        reprojected_data = clipped_data.rio.reproject(to_crs, resolution=(x_res, y_res))
    else:
        reprojected_data = clipped_data
    
    return reprojected_data

In [ ]:
# Loading raster data
cez_raster_2015 = clip_and_reproject(rxr.open_rasterio(chernobyl_path(2015)), clip_geometry=cez.geometry)

# Plot the data
fig, ax = plt.subplots(1, figsize=(6, 3))
cez_raster_2015.plot(cmap="viridis", ax=ax);

In [ ]:
# Load geograph from the raster data (construction takes ~10s)
cez_graph_2015 = GeoGraph(data=cez_raster_2015.data, 
                          transform=cez_raster_2015.rio.transform(), 
                          mask=cez_raster_2015.data > 0,
                          crs=UTM35N, 
                          connectivity=8)

# Visualize geograph
fig, ax = plt.subplots(1, figsize=(6,3))
cez_graph_2015.df.plot("class_label", ax=ax, edgecolor="grey");

Next, we can calculate the metrics for this GeoGraph. 

# 3. Investigating metrics

Let us now investigate a couple of the standard metrics that are used in landscape ecology.
By default GeoGraph always uses the metrics of the CRS system that the data is in, so
meters in our case of UTM35N.

Conveniently, GeoGraph includes the units for us under the "unit" parameter.

### 3.1 Landscape level metrics

In [ ]:
# Calculating the landscape total area
cez_graph_2015.get_metric("total_area", class_value=11)

In [ ]:
# Calculating the landscape shannon index
cez_graph_2015.get_metric("shannon_diversity_index")

In [ ]:
# Calculating the landscape simpson diversity index
cez_graph_2015.get_metric("simpson_diversity_index")

#### 3.1.1 Comparison to pylandstats landscape level metrics


To convince us that these metrics are sensible, we compare to the python implementation [pylandstats](https://pylandstats.readthedocs.io/en/latest/index.html) of the popular FRAGSTATS package.
As we will see, GeoGraphs allow us to compute (almost - currently only limited by our time for implementing them) any metric that pylandstas supports. We will also see that the metrics agree, if we note the Caveat that pylandtats uses hecatres as base unit for computations, while GeoGraph uses the unit of the CRS of the underlying data (meters in our case of `UTM35N`).

In [ ]:
import pylandstats as pls

cez_landscape_2015 = pls.Landscape(cez_raster_2015.data.squeeze(), res=(300,300) , nodata=0)

In [ ]:
print("Shannon diversity index:")
print(f"\t Pylandstats: {cez_landscape_2015.shannon_diversity_index()}")
print(f"\t Geograph:    {cez_graph_2015.get_metric('shannon_diversity_index').value}")

In [ ]:
print("Total area:")
print(f"\t Pylandstats: {cez_landscape_2015.total_area()} ha")
print(f"\t Geograph:    {cez_graph_2015.get_metric('total_area').value} meter^2")

### 3.2 Class level metrics

On top of landscape level metrics, we might be interested in how different landcover classes are distributed across the landscape. For this purpose we can use class level metrics:

For a geograph, we can request the metrics individually via the `get_metrics` method as before. If we want to have several metrics for multiple classes, we can also take the `get_class_metrics` shortcut. 
GeoGraph caches the computations under the hood to speed up future computations.

In [ ]:
cez_graph_2015.get_metric("proportion_of_landscape", class_value=10)

In [ ]:
cez_graph_2015.get_class_metrics(names = ["num_patches", "effective_mesh_size"], classes=[10,11,210])

To get all metrics, simply omit the arguments.

In [ ]:
cez_graph_2015.get_class_metrics()

Let's compare with pylandstats to check the correctness of the metrics:

In [ ]:
pls_class_metrics = [
    'number_of_patches',
     'area_mn',
     'total_area',
     'proportion_of_landscape',
     'patch_density',
     'largest_patch_index',
     'total_edge',
     'effective_mesh_size'
]

pls_metrics = cez_landscape_2015.compute_class_metrics_df(metrics=pls_class_metrics, classes=cez_graph_2015.classes)
pls_metrics

As we can see, the metrics agree if we account for the conversion between `metres` and `hectares` and `fractions` in GeoGraph versus `percentages` in pylandstats.

### 3.2 Patch level metrics

If we're interested in the more fine-grained distribution of metrics on the patch level, 
GeoGraph can also calculate several standard patch-level metrics of landscape ecology for us.

In [ ]:
patch_metrics = cez_graph_2015.get_patch_metrics()
patch_metrics  # again, metrics are in units of UTM35N (meter) 

# Classes of interest
classes_of_interest = patch_metrics["class_label"].isin([10, 11, 210])

We can now use this metrics to investigate the underlying patch-area distribution for example

In [ ]:
import seaborn as sns
sns.displot(patch_metrics["area"])
plt.xlabel("Area [meter^2]")
plt.semilogx();

We can also investigate the dimensionless metrics `shape_index`, `fractal_dimension`.

In [ ]:
sns.displot(patch_metrics["fractal_dimension"])
sns.displot(patch_metrics["shape_index"]);

Further, we can easily analyse distributions for certain classes.

In [ ]:
sns.displot(patch_metrics[classes_of_interest], x="fractal_dimension", hue="class_label", palette="colorblind")
sns.displot(patch_metrics[classes_of_interest], x="perimeter_area_ratio", hue="class_label", palette="colorblind");